In [0]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-23 06:31:33--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv’

customer_churn.csv  100%[===================>] 668.81K  --.-KB/s    in 0.06s   

2019-04-23 06:31:33 (10.6 MB/s) - ‘customer_churn.csv’ saved [684858/684858]



In [0]:
from fastai.tabular import *

In [0]:
import pandas as pd

In [0]:
df=pd.read_csv('customer_churn.csv')

In [0]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
procs = [FillMissing, Categorify, Normalize]
valid_idx = range(len(df)-2000, len(df))

In [0]:
dep_var = 'Exited'
cat_names = ['Geography','Gender','HasCrCard','IsActiveMember']
cont_names = ['CreditScore','Age','Tenure','Balance','NumOfProducts','EstimatedSalary']

In [0]:
data = TabularDataBunch.from_df('',df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names,cont_names=cont_names)

In [0]:
print(data.train_ds.cont_names)

['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']


In [0]:
(cat_x,cont_x),y = next(iter(data.train_dl))

In [0]:
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[1 2 2 1]
 [1 2 1 2]
 [3 1 2 2]
 [3 1 1 1]
 [1 1 2 2]]
[[-0.456192  0.101047  0.342275  0.684421  0.809463 -0.928719]
 [-0.993137 -0.755181 -1.391202 -1.230944  0.809463  1.607363]
 [-0.373585 -0.279499 -1.391202 -0.524837 -0.905501 -0.467101]
 [ 0.390528 -0.564908 -1.044506 -1.230944 -0.905501 -1.262802]
 [-0.549125 -0.279499  1.382361 -1.230944  0.809463 -1.617455]]
[0 0 0 1 0]


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.438578,0.383453,0.834000,00:01
1,0.370618,0.355171,0.858000,00:01
2,0.344101,0.350290,0.854500,00:01


In [0]:
learn.predict(df.iloc[0])

(Category 0, tensor(0), tensor([0.6843, 0.3157]))